# Params.py

In [1]:
import argparse
import torch

def parse_args():
  parser = argparse.ArgumentParser('Model Description')
  parser.add_argument('--lr', default=1e-3, type=float, help='learning rate')
  parser.add_argument('--batch', default=256, type=int, help='batch size')
  parser.add_argument('--sslbatch', default=4096, type=int, help='SSL batch size')
  parser.add_argument('--reg', default=1e-5, type=float, help='weight decay regularizer')
  parser.add_argument('--epoch', default=100, type=int, help='number of epochs')
  parser.add_argument('--decayRate', default=0.96, type=float, help='decay rate for learning rate')
  parser.add_argument('--save_path', default='tem', help='file name to save model and training record')
  parser.add_argument('--latdim', default=32, type=int, help='embedding size')
  parser.add_argument('--rank', default=4, type=int, help='embedding size')
  parser.add_argument('--memosize', default=2, type=int, help='memory size')
  parser.add_argument('--n_factors', default=4, type=int, help='Number of factors to disentangle the original embed-size representation.')
  parser.add_argument('--n_iterations', default=2, type=int, help='Number of iterations to perform the routing mechanism.')
  parser.add_argument('--sampNum', default=40, type=int, help='batch size for sampling')
  parser.add_argument('--att_head', default=2, type=int, help='number of attention heads')
  parser.add_argument('--gnn_layer', default=2, type=int, help='number of gnn layers')
  parser.add_argument('--hyperNum', default=128, type=int, help='number of hyper edges')
  parser.add_argument('--trnNum', default=10000, type=int, help='number of training instances per epoch')
  parser.add_argument('--load_model', default=None, help='model name to load')
  parser.add_argument('--shoot', default=20, type=int, help='K of top k')
  parser.add_argument('--data', default='yelp', type=str, help='name of dataset')
  parser.add_argument('--target', default='buy', type=str, help='target behavior to predict on')
  parser.add_argument('--deep_layer', default=0, type=int, help='number of deep layers to make the final prediction')
  parser.add_argument('--mult', default=100, type=float, help='multiplier for the result')
  #parser.add_argument('--keepRate', default=0.5, type=float, help='rate for dropout')
  parser.add_argument('--droprate', default=0.5, type=float, help='rate for dropout')
  parser.add_argument('--slot', default=5, type=float, help='length of time slots')
  parser.add_argument('--graphSampleN', default=15000, type=int, help='use 25000 for training and 200000 for testing, empirically')
  parser.add_argument('--divSize', default=10000, type=int, help='div size for smallTestEpoch')
  parser.add_argument('--tstEpoch', default=3, type=int, help='number of epoch to test while training')
  parser.add_argument('--subUsrSize', default=10, type=int, help='number of item for each sub-user')
  parser.add_argument('--subUsrDcy', default=0.9, type=float, help='decay factor for sub-users over time')
  parser.add_argument('--leaky', default=0.5, type=float, help='slope for leaky relu')
  parser.add_argument('--hyperReg', default=1e-4, type=float, help='regularizer for hyper connections')
  parser.add_argument('--temp', default=1, type=float, help='temperature in ssl loss')
  parser.add_argument('--ssl_reg', default=1e-4, type=float, help='reg weight for ssl loss')
  parser.add_argument('--percent', default=0.0, type=float, help='percent of noise for noise robust test')
  parser.add_argument('--tstNum', default=99, type=int, help='Numer of negative samples while testing, -1 for all negatives')
  parser.add_argument('--seed', default=10, type=int, help='Random seed')

  #	return parser.parse_args()
  return parser

args, _ = parse_args().parse_known_args()
args.decay_step = args.trnNum/args.batch
if torch.cuda.is_available():
	args.device = "cuda"
else:
	args.device = "cpu"


# DataHandler.py

In [2]:
import scipy.sparse as sp
from scipy.sparse import coo_matrix, csr_matrix
import torch
import numpy as np
import pickle
#from Params import args


class DataHandler:
  def __init__(self):
    if args.data == 'yelp':
        predir = '/content/drive/MyDrive/HCCF_data/yelp/'
    elif args.data == 'ml10m':
        predir = '/content/drive/MyDrive/HCCF_data/ml10m'
    elif args.data == 'amazon':
        predir = '/content/drive/MyDrive/HCCF_data/amazon/'
    self.predir = predir
    self.trnfile = predir + 'trnMat.pkl'
    self.tstfile = predir + 'tstMat.pkl'

  def LoadData(self):
    if args.percent > 1e-8:
      print('noised')
      with open(self.predir + 'noise_%.2f' % args.percent, 'rb') as fs:
        trnMat = (pickle.load(fs) != 0).astype(np.float32)
    else:
      with open(self.trnfile, 'rb') as fs:
        trnMat = (pickle.load(fs) != 0).astype(np.float32)
    # test set
    with open(self.tstfile, 'rb') as fs:
      tstMat = pickle.load(fs)
      # tstMat = (pickle.load(fs) != 0).astype(np.float32)
    tstLocs = [None] * tstMat.shape[0]
    tstUsrs = set()
    for i in range(len(tstMat.data)):
      row = tstMat.row[i]
      col = tstMat.col[i]
      if tstLocs[row] is None:
        tstLocs[row] = list()
      tstLocs[row].append(col)
      tstUsrs.add(row)
    tstUsrs = np.array(list(tstUsrs))

    self.trnMat = trnMat
    self.tstLocs = tstLocs
    self.tstUsrs = tstUsrs
    args.user, args.item = self.trnMat.shape
    self.prepareGlobalData()

  def prepareGlobalData(self):
    adj = self.trnMat
    adj = (adj != 0).astype(np.float32)
    self.labelP = np.squeeze(np.array(np.sum(adj, axis=0)))
    tpadj = transpose(adj)
    adjNorm = np.reshape(np.array(np.sum(adj, axis=1)), [-1])
    tpadjNorm = np.reshape(np.array(np.sum(tpadj, axis=1)), [-1])
    for i in range(adj.shape[0]):
      for j in range(adj.indptr[i], adj.indptr[i+1]):
        adj.data[j] /= adjNorm[i]
    for i in range(tpadj.shape[0]):
      for j in range(tpadj.indptr[i], tpadj.indptr[i+1]):
        tpadj.data[j] /= tpadjNorm[i]
    self.adj = adj
    self.tpadj = tpadj

def transpose(mat):
	coomat = coo_matrix(mat)
	return csr_matrix(coomat.transpose())

def negSamp(temLabel, sampSize, nodeNum):
	negset = [None] * sampSize
	cur = 0
	while cur < sampSize:
		rdmItm = np.random.choice(nodeNum)
		if temLabel[rdmItm] == 0:
			negset[cur] = rdmItm
			cur += 1
	return negset

def transToLsts(mat, mask=False, norm=False):
  shape = torch.Size(mat.shape)
  mat = sp.coo_matrix(mat)
  indices = torch.from_numpy(np.vstack((mat.row, mat.col)).astype(np.int64))
  data = mat.data
  
  if norm:
    rowD = np.squeeze(np.array(1 / (np.sqrt(np.sum(mat, axis=1) + 1e-8) + 1e-8)))
    colD = np.squeeze(np.array(1 / (np.sqrt(np.sum(mat, axis=0) + 1e-8) + 1e-8)))
    for i in range(len(mat.data)):
      row = indices[0, i]
      col = indices[1, i]
      data[i] = data[i] * rowD[row] * colD[col]
	# half mask
  if mask:
    spMask = (np.random.uniform(size=data.shape) > 0.5) * 1.0
    data = data * spMask

  if indices.shape[0] == 0:
    indices = np.array([[0, 0]], dtype=np.int32)
    data = np.array([0.0], np.float32)

  data = torch.from_numpy(data)
	#a =torch.sparse.FloatTensor(indices, values, shape).to(torch.float32).cuda()
  return indices, data, shape


# TimeLogger.py

In [3]:
import datetime

logmsg = ''
timemark = dict()
saveDefault = False
def log(msg, save=None, oneline=False):
	global logmsg
	global saveDefault
	time = datetime.datetime.now()
	tem = '%s: %s' % (time, msg)
	if save != None:
		if save:
			logmsg += tem + '\n'
	elif saveDefault:
		logmsg += tem + '\n'
	if oneline:	
		print(tem, end='\r')
	else:
		print(tem)

def marktime(marker):
	global timemark
	timemark[marker] = datetime.datetime.now()

def SpentTime(marker):
	global timemark
	if marker not in timemark:
		msg = 'LOGGER ERROR, marker', marker, ' not found'
		tem = '%s: %s' % (time, msg)
		print(tem)
		return False
	return datetime.datetime.now() - timemark[marker]

def SpentTooLong(marker, day=0, hour=0, minute=0, second=0):
	global timemark
	if marker not in timemark:
		msg = 'LOGGER ERROR, marker', marker, ' not found'
		tem = '%s: %s' % (time, msg)
		print(tem)
		return False
	return datetime.datetime.now() - timemark[marker] >= datetime.timedelta(days=day, hours=hour, minutes=minute, seconds=second)

if __name__ == '__main__':
	log('')


2022-09-07 13:10:28.828276: 


# Model.py

In [4]:
import torch
import torch.nn as nn
#from Params import args

torch.manual_seed(666)

def LeakyRelu(data):
  #global leaky
  ret = torch.maximum(args.leaky*data, data)
  return ret

class FC(nn.Module):
  def __init__(self, inputDim, outDim, Bias = False, actFunc = None):
    super(FC,self).__init__()
    initializer = nn.init.xavier_normal_
    self.W_fc = nn.Parameter(initializer(torch.empty(inputDim, outDim).cuda()))

  def forward(self, inp, droprate = 0):
    #W = self.W_fc.weight
    fc1 = inp @ self.W_fc
    ret = fc1
    ret = LeakyRelu(ret)
    return ret

class hyperPropagate(nn.Module):
  def __init__(self,inputdim):
    super(hyperPropagate, self).__init__()
    self.inputdim = inputdim
    self.fc1 = FC(self.inputdim,args.hyperNum,actFunc = 'leakyRelu').cuda()
    self.fc2 = FC(self.inputdim,args.hyperNum,actFunc = 'leakyRelu').cuda()
    self.fc3 = FC(self.inputdim,args.hyperNum,actFunc = 'leakyRelu').cuda()
    #self.actFunc = nn.LeakyReLU(negative_slope=args.leaky) 

  def forward(self,lats,adj):
    lat1 = LeakyRelu(torch.transpose(adj,0,1) @ lats) #shape adj:user,hyperNum lats:user,latdim lat1:hypernum,latdim
    lat2 = torch.transpose(self.fc1(torch.transpose(lat1,0,1)),0,1) + lat1 #shape hypernum,latdim
    lat3 = torch.transpose(self.fc2(torch.transpose(lat2,0,1)),0,1) + lat2
    lat4 = torch.transpose(self.fc3(torch.transpose(lat3,0,1)),0,1) + lat3
    ret = adj @ lat4
    ret = LeakyRelu(ret)
    return ret

class weight_trans(nn.Module):
  def __init__(self):
    super(weight_trans, self).__init__()
    initializer = nn.init.xavier_normal_
    self.W = nn.Parameter(initializer(torch.empty(args.latdim, args.latdim).cuda()))

  def forward(self,normalize):
    ret = normalize @ self.W
    return ret

class HCCF(nn.Module):
  def __init__(self, adj_py, tpAdj_py):
    super(HCCF, self).__init__()
    initializer = nn.init.xavier_normal_
    self.uEmbed0 = nn.Parameter(initializer(torch.empty(args.user, args.latdim).cuda()))
    self.iEmbed0 = nn.Parameter(initializer(torch.empty(args.item, args.latdim).cuda()))
    self.uhyper = nn.Parameter(initializer(torch.empty(args.latdim, args.hyperNum).cuda()))
    self.ihyper = nn.Parameter(initializer(torch.empty(args.latdim, args.hyperNum).cuda()))

    self.adj = adj_py.cuda()#shape user,item
    self.tpadj = tpAdj_py.cuda()#shape item,user

    self.hyperULat_layers = nn.ModuleList()
    self.hyperILat_layers = nn.ModuleList()
    self.weight_layers = nn.ModuleList()
    
    for i in range(args.gnn_layer):
      self.hyperULat_layers.append(hyperPropagate(args.hyperNum)) #shape hyperNum,hyperNum
      self.hyperILat_layers.append(hyperPropagate(args.hyperNum)) #shape hyperNum,hyperNum
      self.weight_layers.append(weight_trans())


  def messagePropagate(self, lats, adj):
    return LeakyRelu(torch.sparse.mm(adj, lats))

  def calcSSL(self, hyperLat, gnnLat):
    posScore = torch.exp(torch.sum(hyperLat * gnnLat, dim = 1) / args.temp)
    negScore = torch.sum(torch.exp(gnnLat @ torch.transpose(hyperLat, 0, 1) / args.temp), dim = 1)
    uLoss = torch.sum(-torch.log(posScore / (negScore + 1e-8) + 1e-8))
    return uLoss

  def Regularize(self, reg, method = 'L2'):
    ret = 0.0
    for i in range(len(reg)):
        ret += torch.sum(torch.square(reg[i]))
    return ret

  def edgeDropout(self, mat, drop):
    def dropOneMat(mat):
      indices = mat._indices().cpu()
      values = mat._values().cpu()
      shape = mat.shape
      newVals = nn.functional.dropout(values, p = drop)
      return torch.sparse.FloatTensor(indices, newVals, shape).to(torch.float32).cuda()
    return dropOneMat(mat)

  def forward_test(self):
    uEmbed0 = self.uEmbed0
    iEmbed0 = self.iEmbed0
    uhyper = self.uhyper
    ihyper = self.ihyper

    uuHyper = uEmbed0 @ uhyper#shape user,hyperNum
    iiHyper = iEmbed0 @ ihyper#shape item,hyperNum

    ulats = [uEmbed0]
    ilats = [iEmbed0]

    for i in range(args.gnn_layer):
      ulat = self.messagePropagate(ilats[-1], self.edgeDropout(self.adj, drop = 0))
      ilat = self.messagePropagate(ulats[-1], self.edgeDropout(self.tpadj, drop = 0))
      hyperULat = self.hyperULat_layers[i](ulats[-1],nn.functional.dropout(uuHyper, p = 0))
      hyperILat = self.hyperILat_layers[i](ilats[-1],nn.functional.dropout(iiHyper, p = 0))

      ulats.append(ulat + hyperULat + ulats[-1])
      ilats.append(ilat + hyperILat + ilats[-1])

    ulat = sum(ulats)
    ilat = sum(ilats)
    return ulat, ilat

  def forward(self, uids, iids, droprate = args.droprate):
    uEmbed0 = self.uEmbed0
    iEmbed0 = self.iEmbed0
    uhyper = self.uhyper
    ihyper = self.ihyper
    gnnULats = []
    gnnILats = []
    hyperULats = []
    hyperILats = []

    ulats = [uEmbed0]
    ilats = [iEmbed0]
    for i in range(args.gnn_layer):
      ulat = self.messagePropagate(ilats[-1], self.edgeDropout(self.adj, drop = droprate))
      ilat = self.messagePropagate(ulats[-1], self.edgeDropout(self.tpadj, drop = droprate))
      hyperULat = self.hyperULat_layers[i](ulats[-1],nn.functional.dropout(uEmbed0 @ uhyper, p = droprate))# / (1 - droprate))
      hyperILat = self.hyperILat_layers[i](ilats[-1],nn.functional.dropout(iEmbed0 @ ihyper, p = droprate))#/ (1 - droprate) )

      gnnULats.append(ulat)
      gnnILats.append(ilat)
      hyperULats.append(hyperULat)
      hyperILats.append(hyperILat)

      ulats.append(ulat + hyperULat + ulats[-1])
      ilats.append(ilat + hyperILat + ilats[-1])

    ulat = sum(ulats)
    ilat = sum(ilats)
    pckUlat = torch.index_select(ulat, 0, uids)
    pckIlat = torch.index_select(ilat, 0, iids)
    preds = torch.sum(pckUlat * pckIlat, dim=-1)

    sslloss = 0
    uniqUids = torch.unique(uids)
    uniqIids = torch.unique(iids)

    for i in range(len(hyperULats)):
      pckHyperULat = self.weight_layers[i](torch.nn.functional.normalize(torch.index_select(hyperULats[i], 0, uniqUids), p=2, dim=1))# @ self.weight_layers[i].weight
      pckGnnULat = torch.nn.functional.normalize(torch.index_select(gnnULats[i], 0, uniqUids), p=2, dim=1)
      pckhyperILat = self.weight_layers[i](torch.nn.functional.normalize(torch.index_select(hyperILats[i], 0, uniqIids), p=2, dim=1))# @ self.weight_layers[i].weight
      pckGnnILat = torch.nn.functional.normalize(torch.index_select(gnnILats[i], 0, uniqIids), p=2, dim=1)
      uLoss = self.calcSSL(pckHyperULat, pckGnnULat)
      iLoss = self.calcSSL(pckhyperILat, pckGnnILat)
      sslloss += uLoss + iLoss

    return preds, sslloss, self.Regularize([uEmbed0,iEmbed0,uhyper,ihyper])

# HCCF.py

In [5]:
import torch
import numpy as np

# from Model import HCCF
# from DataHandler import DataHandler, negSamp, transToLsts, transpose
# from Params import args
# from TimeLogger import log

torch.manual_seed(666)
np.random.seed(666)

class hccf():
    def __init__(self,handler):
        self.handler = handler
        self.handler.LoadData()

        adj = handler.trnMat
        idx, data, shape = transToLsts(adj, norm=True)
        self.adj_py = torch.sparse.FloatTensor(idx, data, shape).to(torch.float32).cuda()
        idx, data, shape = transToLsts(transpose(adj), norm=True)
        self.tpAdj_py = torch.sparse.FloatTensor(idx, data, shape).to(torch.float32).cuda()

        self.curepoch = 0
        self.metrics = dict()
        mets = ['Loss', 'preLoss', 'Recall', 'NDCG']
        for met in mets:
          self.metrics['Train' + met] = list()
          self.metrics['Test' + met] = list()

    def preparemodel(self):
        self.model = HCCF(self.adj_py, self.tpAdj_py).cuda()
        self.opt = torch.optim.Adam(params = self.model.parameters(), lr=args.lr)
        self.scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer = self.opt, gamma=args.decayRate)
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                print(name, param.shape)

    def sampleTrainBatch(self, batIds, labelMat):
        temLabel = labelMat[batIds].toarray()
        batch = len(batIds)
        temlen = batch * 2 * args.sampNum
        uLocs = [None] * temlen
        iLocs = [None] * temlen
        cur = 0
        for i in range(batch):
            posset = np.reshape(np.argwhere(temLabel[i]!=0), [-1])
            sampNum = min(args.sampNum, len(posset))
            if sampNum == 0:
                poslocs = [np.random.choice(args.item)]
                neglocs = [poslocs[0]]
            else:
                poslocs = np.random.choice(posset, sampNum)
                neglocs = negSamp(temLabel[i], sampNum, args.item)
            for j in range(sampNum):
                posloc = poslocs[j]
                negloc = neglocs[j]
                uLocs[cur] = uLocs[cur+temlen//2] = batIds[i]
                iLocs[cur] = posloc
                iLocs[cur+temlen//2] = negloc
                cur += 1
        uLocsa = uLocs[:cur] + uLocs[temlen//2: temlen//2 + cur]
        iLocsa = iLocs[:cur] + iLocs[temlen//2: temlen//2 + cur]
        
        return torch.Tensor(uLocsa).cuda(), torch.Tensor(iLocsa).cuda()

    def trainEpoch(self):
        args.actFunc = 'leakyRelu'

        num = args.user
        #randomly select args.trnNum users(10,000), from args.user(29,601 amazon), as input.
        sfIds = np.random.permutation(args.user)[:args.trnNum]
        epochLoss, epochPreLoss, epochsslloss, epochregloss = [0] * 4
        num = len(sfIds)
        steps = int(np.ceil(num / args.batch))
        self.model.train()

        for i in range(steps):
            st = i * args.batch
            ed = min((i+1) * args.batch, num)
            batIds = sfIds[st: ed]

            uLocs, iLocs = self.sampleTrainBatch(batIds, self.handler.trnMat)
            
            preds, sslloss, regularize = self.model(uLocs.int(),iLocs.int())
            sampNum = uLocs.shape[0] // 2
            posPred = preds[:sampNum]
            negPred = preds[sampNum:sampNum * 2]
            preLoss = torch.sum(torch.maximum(torch.Tensor([0.0]).to(args.device), 1.0 - (posPred - negPred))) / args.batch
            sslloss = args.ssl_reg * sslloss
            regLoss = args.reg * regularize

            loss = preLoss + regLoss + sslloss

            self.opt.zero_grad()
            loss.backward()
            self.opt.step()
            if i % args.decay_step == 0:
                self.scheduler.step()

            epochLoss += loss
            epochPreLoss += preLoss
            epochregloss += args.reg * regularize
            epochsslloss += args.ssl_reg * sslloss
            #log('Step %d/%d: loss = %.2f, regLoss = %.2f         ' % (i, steps, loss, regLoss), save=False, oneline=False)

        ret = dict()
        ret['Loss'] = epochLoss / steps
        ret['preLoss'] = epochPreLoss / steps
        ret['sslLoss'] = epochsslloss / steps
        ret['regLoss'] = epochregloss / steps

        return ret

    def testEpoch(self):
      self.model.eval()
      with torch.no_grad():
        epochRecall, epochNdcg = [0] * 2
        ids = self.handler.tstUsrs
        num = len(ids)
        tstBat = args.batch
        steps = int(np.ceil(num / tstBat))
        tstNum = 0
        ulat, ilat = self.model.forward_test()
        for i in range(steps):
            st = i * tstBat
            ed = min((i+1) * tstBat, num)
            batIds = ids[st: ed]
            trnPosMask = self.handler.trnMat[batIds].toarray()
            toplocs = self.tstPred(batIds, trnPosMask, ulat, ilat)
            recall, ndcg = self.calcRes(toplocs, self.handler.tstLocs, batIds)
            epochRecall += recall
            epochNdcg += ndcg
            #log('Steps %d/%d: recall = %.2f, ndcg = %.2f          ' % (i, steps, recall, ndcg), save=False, oneline=False)
        ret = dict()
        ret['Recall'] = epochRecall / num
        ret['NDCG'] = epochNdcg / num
      return ret

    def tstPred(self, batIds, trnPosMask, ulat, ilat):
      pckUlat = torch.index_select(ulat, 0, torch.Tensor(batIds).int().to(args.device))
      allPreds = pckUlat @ torch.transpose(ilat, 0, 1)
      allPreds = allPreds.cpu().detach().numpy() * (1 - trnPosMask) - trnPosMask * 1e8
      vals, locs = torch.topk(torch.tensor(allPreds), args.shoot)
      return locs

    def calcRes(self, topLocs, tstLocs, batIds):
        assert topLocs.shape[0] == len(batIds)
        allRecall = allNdcg = 0
        recallBig = 0
        ndcgBig =0
        for i in range(len(batIds)):
            temTopLocs = list(topLocs[i])
            temTstLocs = tstLocs[batIds[i]]
            tstNum = len(temTstLocs)
            maxDcg = np.sum([np.reciprocal(np.log2(loc + 2)) for loc in range(min(tstNum, args.shoot))])
            recall = dcg = 0
            for val in temTstLocs:
                if val in temTopLocs:
                    recall += 1
                    dcg += np.reciprocal(np.log2(temTopLocs.index(val) + 2))
            recall = recall / tstNum
            ndcg = dcg / maxDcg
            allRecall += recall
            allNdcg += ndcg
        return allRecall, allNdcg

    def loadModel(self, loadPath):
        loadPath = loadPath
        checkpoint = torch.load(loadPath)
        self.model = checkpoint['model']
        self.curepoch = checkpoint['epoch']+1
        self.metrics = checkpoint['metrics']

    def saveHistory(self):

        savePath = r'./Model/' + args.data  + r'.pth'
        params = {
            'epoch' : self.curepoch,
            'model' : self.model,
            'metrics' : self.metrics,
        }
        torch.save(params, savePath)


    def run(self):
        self.preparemodel()
        log('Model Prepared')
        if args.load_model != None:
            self.loadModel(args.load_model)
            stloc = self.curepoch
        else:
            stloc = 0

        for ep in range(stloc, args.epoch):
            test = (ep % args.tstEpoch == 0)
            reses = self.trainEpoch()
            #print(self.model.hyperULat_layers[0].fc1.W_fc.weight)
            log(self.makePrint('Train', ep, reses, test))
            if test:
                reses = self.testEpoch()
                log(self.makePrint('Test', ep, reses, test))
            if ep % args.tstEpoch == 0:
                self.saveHistory()
            print()
            self.curepoch = ep
        reses = self.testEpoch()
        log(self.makePrint('Test', args.epoch, reses, True))
        self.saveHistory()

    def makePrint(self, name, ep, reses, save):
      ret = 'Epoch %d/%d, %s: ' % (ep, args.epoch, name)
      for metric in reses:
            val = reses[metric]
            ret += '%s = %.4f, ' % (metric, val)
            tem = name + metric
            if save and tem in self.metrics:
                self.metrics[tem].append(val)
      ret = ret[:-2] + '  '
      return ret

if __name__ == '__main__':
    handler = DataHandler()
    handler.LoadData()
    model=hccf(handler)
    model.run()

uEmbed0 torch.Size([29601, 32])
iEmbed0 torch.Size([24734, 32])
uhyper torch.Size([32, 128])
ihyper torch.Size([32, 128])
hyperULat_layers.0.fc1.W_fc torch.Size([128, 128])
hyperULat_layers.0.fc2.W_fc torch.Size([128, 128])
hyperULat_layers.0.fc3.W_fc torch.Size([128, 128])
hyperULat_layers.1.fc1.W_fc torch.Size([128, 128])
hyperULat_layers.1.fc2.W_fc torch.Size([128, 128])
hyperULat_layers.1.fc3.W_fc torch.Size([128, 128])
hyperILat_layers.0.fc1.W_fc torch.Size([128, 128])
hyperILat_layers.0.fc2.W_fc torch.Size([128, 128])
hyperILat_layers.0.fc3.W_fc torch.Size([128, 128])
hyperILat_layers.1.fc1.W_fc torch.Size([128, 128])
hyperILat_layers.1.fc2.W_fc torch.Size([128, 128])
hyperILat_layers.1.fc3.W_fc torch.Size([128, 128])
weight_layers.0.W torch.Size([32, 32])
weight_layers.1.W torch.Size([32, 32])
2022-09-07 13:10:54.467882: Model Prepared
2022-09-07 13:11:04.471152: Epoch 0/100, Train: Loss = 36.5366, preLoss = 20.3680, sslLoss = 0.0016, regLoss = 0.0024  
2022-09-07 13:11:42.32135